# How to make targeted offers to customers?

This tutorial includes everything you need to set up IBM Decision Optimization CPLEX Modeling for Python (DOcplex), build a Mathematical Programming model, and get its solution by solving the model with IBM ILOG CPLEX Optimizer.

When you finish this tutorial, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook is part of [Prescriptive Analytics for Python](http://ibmdecisionoptimization.github.io/docplex-doc/)
>
>It requires either an [installation of CPLEX Optimizers](http://ibmdecisionoptimization.github.io/docplex-doc/getting_started.html) or it can be run on [IBM Cloud Pak for Data as a Service](https://www.ibm.com/products/cloud-pak-for-data/as-a-service/) (Sign up for a [free IBM Cloud account](https://dataplatform.cloud.ibm.com/registration/stepone?context=wdp&apps=all>)
and you can start using `IBM Cloud Pak for Data as a Service` right away).
>
> CPLEX is available on <i>IBM Cloud Pack for Data</i> and <i>IBM Cloud Pak for Data as a Service</i>:
>    - <i>IBM Cloud Pak for Data as a Service</i>: Depends on the runtime used:
>         - <i>Python 3.x</i> runtime: Community edition
>         - <i>Python 3.x + DO</i> runtime: full edition
>    - <i>Cloud Pack for Data</i>: Community edition is installed by default. Please install `DO` addon in `Watson Studio Premium` for the full edition


Table of contents:

-  [Describe the business problem](#Describe-the-business-problem)
*  [How decision optimization (prescriptive analytics) can help](#How-decision-optimization-can-help)
*  [Prepare the data](#Prepare-the-data)
*  [Use decision optimization](#Use-IBM-Decision-Optimization-CPLEX-Modeling-for-Python)
    *  [Step 1: Import the library](#Step-1:-Import-the-library)
    -  [Step 2: Set up the prescriptive model](#Step-2:-Set-up-the-prescriptive-model)
        * [Define the decision variables](#Define-the-decision-variables)
        * [Set up the constraints](#Set-up-the-constraints)
        * [Express the objective](#Express-the-objective)
        * [Solve the model](#Solve-the-model)
    *  [Step 3: Analyze the solution and run an example analysis](#Step-3:-Analyze-the-solution)


## Describe the business problem
* The Self-Learning Response Model (SLRM) node enables you to build a model that you can continually update. Such updates are useful in building a model that assists with predicting which offers are most appropriate for customers and the probability of the offers being accepted. These sorts of models are most beneficial in customer relationship management, such as marketing applications or call centers.
* This example is based on a fictional banking company. 
* The marketing department wants to achieve more profitable results in future campaigns by matching the right offer of financial services to each customer. 
* Specifically, the datascience department identified the characteristics of customers who are most likely to respond favorably based on previous offers and responses and to promote the best current offer based on the results and now need to compute the best offerig plan.
<br>

A set of business constraints have to be respected:

* We have a limited budget to run a marketing campaign based on "gifts", "newsletter", "seminar"...
* We want to determine which is the best way to contact the customers.
* We need to identify which customers to contact.

## How decision optimization can help

* Prescriptive analytics technology recommends actions based on desired outcomes, taking into account specific scenarios, resources, and knowledge of past and current events. This insight can help your organization make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

+ For example:
    + Automate complex decisions and trade-offs to better manage limited resources.
    + Take advantage of a future opportunity or mitigate a future risk.
    + Proactively update recommendations based on changing events.
    + Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

## Prepare the data

The predictions show which offers a customer is most likely to accept, and the confidence that they will accept, depending on each customer’s details.

For example:
<code>(139987, "Pension", 0.13221, "Mortgage", 0.10675)</code> indicates that customer Id=139987 will certainly not buy a _Pension_ as the level is only 13.2%, 
whereas
<code>(140030, "Savings", 0.95678, "Pension", 0.84446)</code> is more than likely to buy _Savings_ and a _Pension_ as the rates are 95.7% and 84.4%.

This data is taken from a SPSS example, except that the names of the customers were modified.

A Python data analysis library, [pandas](http://pandas.pydata.org), is used to store the data. Let's set up and declare the data.

In [1]:
import pandas as pd

names = {
    139987 : "Guadalupe J. Martinez", 140030 : "Michelle M. Lopez", 140089 : "Terry L. Ridgley", 
    140097 : "Miranda B. Roush", 139068 : "Sandra J. Wynkoop", 139154 : "Roland Guérette", 139158 : "Fabien Mailhot", 
    139169 : "Christian Austerlitz", 139220 : "Steffen Meister", 139261 : "Wolfgang Sanger",
    139416 : "Lee Tsou", 139422 : "Sanaa' Hikmah Hakimi", 139532 : "Miroslav Škaroupka", 
    139549 : "George Blomqvist", 139560 : "Will Henderson", 139577 : "Yuina Ohira", 139580 : "Vlad Alekseeva", 
    139636 : "Cassio Lombardo", 139647 : "Trinity Zelaya Miramontes", 139649 : "Eldar Muravyov", 139665 : "Shu T'an", 
    139667 : "Jameel Abdul-Ghani Gerges", 139696 : "Zeeb Longoria Marrero", 139752 : "Matheus Azevedo Melo", 
    139832 : "Earl B. Wood", 139859 : "Gabrielly Sousa Martins", 139881 : "Franca Palermo"}


data = [(139987, "Pension", 0.13221, "Mortgage", 0.10675), (140030, "Savings", 0.95678, "Pension", 0.84446), (140089, "Savings", 0.95678, "Pension", 0.80233), 
                        (140097, "Pension", 0.13221, "Mortgage", 0.10675), (139068, "Pension", 0.80506, "Savings", 0.28391), (139154, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139158, "Pension", 0.13221, "Mortgage", 0.10675),(139169, "Pension", 0.13221, "Mortgage", 0.10675), (139220, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139261, "Pension", 0.13221, "Mortgage", 0.10675), (139416, "Pension", 0.13221, "Mortgage", 0.10675), (139422, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139532, "Savings", 0.95676, "Mortgage", 0.82269), (139549, "Savings", 0.16428, "Pension", 0.13221), (139560, "Savings", 0.95678, "Pension", 0.86779), 
                        (139577, "Pension", 0.13225, "Mortgage", 0.10675), (139580, "Pension", 0.13221, "Mortgage", 0.10675), (139636, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139647, "Savings", 0.28934, "Pension", 0.13221), (139649, "Pension", 0.13221, "Mortgage", 0.10675), (139665, "Savings", 0.95675, "Pension", 0.27248), 
                        (139667, "Pension", 0.13221, "Mortgage", 0.10675), (139696, "Savings", 0.16188, "Pension", 0.13221), (139752, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139832, "Savings", 0.95678, "Pension", 0.83426), (139859, "Savings", 0.95678, "Pension", 0.75925), (139881, "Pension", 0.13221, "Mortgage", 0.10675)]

products = ["Car loan", "Savings", "Mortgage", "Pension"]
productValue = [100, 200, 300, 400]
budgetShare = [0.6, 0.1, 0.2, 0.1]

availableBudget = 500
channels =  pd.DataFrame(data=[("gift", 20.0, 0.20), ("newsletter", 15.0, 0.05), ("seminar", 23.0, 0.30)], columns=["name", "cost", "factor"])

Offers are stored in a [pandas DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [2]:
try: # Python 2
    offers = pd.DataFrame(data=data, index=xrange(0, len(data)), columns=["customerid", "Product1", "Confidence1", "Product2", "Confidence2"])
except: # Python 3
    offers = pd.DataFrame(data=data, index=range(0, len(data)), columns=["customerid", "Product1", "Confidence1", "Product2", "Confidence2"])
offers.insert(0,'name',pd.Series(names[i[0]] for i in data))

Let's customize the display of this data and show the confidence forecast for each customer.

In [3]:
CSS = """
body {
    margin: 0;
    font-family: Helvetica;
}
table.dataframe {
    border-collapse: collapse;
    border: none;
}
table.dataframe tr {
    border: none;
}
table.dataframe td, table.dataframe th {
    margin: 0;
    border: 1px solid white;
    padding-left: 0.25em;
    padding-right: 0.25em;
}
table.dataframe th:not(:empty) {
    background-color: #fec;
    text-align: left;
    font-weight: normal;
}
table.dataframe tr:nth-child(2) th:empty {
    border-left: none;
    border-right: 1px dashed #888;
}
table.dataframe td {
    border: 2px solid #ccf;
    background-color: #f4f4ff;
}
    table.dataframe thead th:first-child {
        display: none;
    }
    table.dataframe tbody th {
        display: none;
    }
"""

In [4]:
from IPython.core.display import HTML
HTML('<style>{}</style>'.format(CSS))

from IPython.display import display
try: 
    display(offers.drop('customerid',1).sort_values(by='name')) #Pandas >= 0.17
except:
    display(offers.drop('customerid',1).sort('name')) #Pandas < 0.17

,name,Product1,Confidence1,Product2,Confidence2
17,Cassio Lombardo,Pension,0.13221,Mortgage,0.10675
7,Christian Austerlitz,Pension,0.13221,Mortgage,0.10675
24,Earl B. Wood,Savings,0.95678,Pension,0.83426
19,Eldar Muravyov,Pension,0.13221,Mortgage,0.10675
6,Fabien Mailhot,Pension,0.13221,Mortgage,0.10675
26,Franca Palermo,Pension,0.13221,Mortgage,0.10675
25,Gabrielly Sousa Martins,Savings,0.95678,Pension,0.75925
13,George Blomqvist,Savings,0.16428,Pension,0.13221
0,Guadalupe J. Martinez,Pension,0.13221,Mortgage,0.10675
21,Jameel Abdul-Ghani Gerges,Pension,0.13221,Mortgage,0.10675


## Use IBM Decision Optimization CPLEX Modeling for Python

Let's create the optimization model to select the best ways to contact customers and stay within the limited budget.

### Step 1: Import the library

Run the following code to import the Decision Optimization CPLEX Modeling library.  The *DOcplex* library contains the two modeling packages, Mathematical Programming (docplex.mp) and Constraint Programming (docplex.cp).

In [5]:
import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

If *cplex* is not installed, you can install CPLEX Community edition.

In [6]:
try:
    import cplex
except:
    raise Exception('Please install CPLEX. See https://pypi.org/project/cplex/')

### Step 2: Set up the prescriptive model
#### Create the model

In [7]:
from docplex.mp.model import Model

mdl = Model(name="marketing_campaign")

#### Define the decision variables
- The integer decision variables `channelVars`, represent whether or not a customer will be made an offer for a particular product via a particular channel.
- The integer decision variable `totaloffers` represents the total number of offers made.
- The continuous variable `budgetSpent` represents the total cost of the offers made.

In [8]:
try: # Python 2
    offersR = xrange(0, len(offers))
    productsR = xrange(0, len(products))
    channelsR = xrange(0, len(channels))
except: # Python 3
    offersR = range(0, len(offers))
    productsR = range(0, len(products))
    channelsR = range(0, len(channels))

channelVars = mdl.binary_var_cube(offersR, productsR, channelsR)
totaloffers = mdl.integer_var(lb=0)
budgetSpent = mdl.continuous_var()

#### Set up the constraints
- Offer only one product per customer.
- Compute the budget and set a maximum on it.
- Compute the number of offers to be made.

In [9]:
# Only 1 product is offered to each customer     
mdl.add_constraints( mdl.sum(channelVars[o,p,c] for p in productsR for c in channelsR) <=1
                   for o in offersR)

mdl.add_constraint( totaloffers == mdl.sum(channelVars[o,p,c] 
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR) )

mdl.add_constraint( budgetSpent == mdl.sum(channelVars[o,p,c]*channels.at[c, "cost"] 
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR) )

# Balance the offers among products   
for p in productsR:
    mdl.add_constraint( mdl.sum(channelVars[o,p,c] for o in offersR for c in channelsR) 
                       <= budgetShare[p] * totaloffers )
            
# Do not exceed the budget
mdl.add_constraint( mdl.sum(channelVars[o,p,c]*channels.at[c, "cost"] 
                            for o in offersR 
                            for p in productsR 
                            for c in channelsR)  <= availableBudget )  

mdl.print_information()

Model: marketing_campaign
 - number of variables: 326
   - binary=324, integer=1, continuous=1
 - number of constraints: 34
   - linear=34
 - parameters: defaults
 - objective: none
 - problem type is: MILP


#### Express the objective

We want to maximize the expected revenue.

In [10]:
mdl.maximize(
    mdl.sum( channelVars[idx,p,idx2] * c.factor * productValue[p]* o.Confidence1  
            for p in productsR 
            for idx,o in offers[offers['Product1'] == products[p]].iterrows()  
            for idx2, c in channels.iterrows())
    +
    mdl.sum( channelVars[idx,p,idx2] * c.factor * productValue[p]* o.Confidence2 
            for p in productsR 
            for idx,o in offers[offers['Product2'] == products[p]].iterrows() 
            for idx2, c in channels.iterrows())
    )

#### Solve the model

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

In [11]:
s = mdl.solve()
assert s, "No Solution !!!"

### Step 3: Analyze the solution

First, let's display the **Optimal Marketing Channel per customer**.

In [12]:
report = [(channels.at[c, "name"], products[p], names[offers.at[o, "customerid"]]) 
          for c in channelsR 
          for p in productsR 
          for o in offersR  if channelVars[o,p,c].solution_value==1]

assert len(report) == totaloffers.solution_value

print("Marketing plan has {0} offers costing {1}".format(totaloffers.solution_value, budgetSpent.solution_value))

report_bd = pd.DataFrame(report, columns=['channel', 'product', 'customer'])
display(report_bd)

Marketing plan has 20 offers costing 364.0


,channel,product,customer
0,newsletter,Car loan,Fabien Mailhot
1,newsletter,Car loan,Christian Austerlitz
2,newsletter,Car loan,Lee Tsou
3,newsletter,Car loan,Sanaa' Hikmah Hakimi
4,newsletter,Car loan,George Blomqvist
5,newsletter,Car loan,Yuina Ohira
6,newsletter,Car loan,Vlad Alekseeva
7,newsletter,Car loan,Cassio Lombardo
8,newsletter,Car loan,Trinity Zelaya Miramontes
9,newsletter,Car loan,Eldar Muravyov



Then let's **focus on seminar**.

In [13]:
display(report_bd[report_bd['channel'] == "seminar"].drop('channel',1))

,product,customer
12,Savings,Terry L. Ridgley
13,Savings,Gabrielly Sousa Martins
14,Mortgage,Miranda B. Roush
15,Mortgage,Miroslav Škaroupka
16,Mortgage,Matheus Azevedo Melo
17,Mortgage,Franca Palermo
18,Pension,Michelle M. Lopez
19,Pension,Will Henderson



## Summary


You learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to formulate a Mathematical Programming model and solve it with CPLEX.


## References
* [CPLEX Modeling for Python documentation](http://ibmdecisionoptimization.github.io/docplex-doc/)
* [IBM Decision Optimization](https://www.ibm.com/analytics/decision-optimization)
* Need help with DOcplex or to report a bug? Please go [here](https://stackoverflow.com/questions/tagged/docplex).
* Contact us at dofeedback@wwpdl.vnet.ibm.com.

Copyright &copy; 2017-2021 IBM. IPLA licensed Sample Materials.